# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [66]:
import torch; import sklearn; from torch import nn

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Preparar dados para pytorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

## Definição do modelo e treinamento

In [67]:
# 3. Definir modelo
import torch.nn.functional as F
class RedeNeural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RedeNeural, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [68]:
# Criar modelo
modelo = RedeNeural(4, 16, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

import copy
modelo_clonado = copy.deepcopy(modelo)

learning_rate = 0.1

# Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=learning_rate)

## Execução do treinamento com optimizer SGD

In [69]:
# Loop de treino
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo(X)             # Forward
    loss = funcao_perda(outputs, y) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente
    optimizer.step()                # Aplica regra de alteração dos parâmetros

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

Época [100/100], Loss: 0.5796
Época [200/100], Loss: 0.5529
Época [300/100], Loss: 0.5569
Época [400/100], Loss: 0.5396
Época [500/100], Loss: 0.3608
Época [600/100], Loss: 0.2632
Época [700/100], Loss: 0.1848
Época [800/100], Loss: 0.1070
Época [900/100], Loss: 0.0917
Época [1000/100], Loss: 0.0821


# Treino com regra de gradiente descendente manual

In [70]:
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo_clonado(X)     # Forward
    loss = funcao_perda(outputs, y) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente

    with torch.no_grad():
        for param in modelo_clonado.parameters():
            param -= learning_rate * param.grad  # regra de atualização de pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000], Loss: {loss.item():.4f}")

Época [100/1000], Loss: 0.6395
Época [200/1000], Loss: 0.6782
Época [300/1000], Loss: 0.6600
Época [400/1000], Loss: 0.6366
Época [500/1000], Loss: 0.6729
Época [600/1000], Loss: 0.6693
Época [700/1000], Loss: 0.6376
Época [800/1000], Loss: 0.6654
Época [900/1000], Loss: 0.6765
Época [1000/1000], Loss: 0.6421


In [71]:
modelo.eval() 

with torch.no_grad():
    # 1. Obter as probabilidades (log-odds)
    outputs = modelo(X)
    
    # 2. Aplicar Sigmoid (para obter probabilidades entre 0 e 1)
    # A função de perda BCEWithLogitsLoss já inclui o Sigmoid no cálculo,
    # mas para a previsão de classes, precisamos aplicá-lo explicitamente.
    probabilities = torch.sigmoid(outputs)
    
    # 3. Classificar: se P > 0.5, é Versicolor (1), caso contrário é Outro (0)
    predictions = (probabilities >= 0.5).float()
    
    # 4. Calcular a Precisão (Accuracy)
    # Comparar previsões com os rótulos reais (y)
    correct = (predictions == y).sum().item()
    total = y.size(0)
    accuracy = correct / total
    
    print(f"\n--- Avaliação do Modelo Treinado ---")
    print(f"Total de amostras: {total}")
    print(f"Acertos: {correct}")
    print(f"Precisão (Accuracy): {accuracy:.4f}")

# Voltar para o modo de treino
modelo.train()


--- Avaliação do Modelo Treinado ---
Total de amostras: 150
Acertos: 148
Precisão (Accuracy): 0.9867


RedeNeural(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)